In [1]:
%run 01_setup.ipynb

23/11/20 13:57:11 WARN SSLSocketFactoryEx: Failed to load OpenSSL. Falling back to the JSSE default.


In [22]:
import time
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, max, min, col, row_number

In [3]:
hw_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- avg_tmpr_c: double (nullable = true)
 |-- avg_tmpr_f: double (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- geoHash: string (nullable = true)
 |-- id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- wthr_date: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [20]:
hw_clear = hw_df.groupBy('id', 'year', 'month') \
    .agg(
        max('avg_tmpr_c').alias('max_t'),
        min('avg_tmpr_c').alias('min_t'),
        (max('avg_tmpr_c') - min('avg_tmpr_c')).alias('t_diff')
    ).where('t_diff > 0')

In [26]:
start_time = time.time()

window = Window.partitionBy('year', 'month').orderBy(col('t_diff').desc())
result = hw_clear.withColumn("rank", row_number().over(window)).where(col("rank").cast("int") < 11)
result.show()

print("Execution time is ", round(time.time() - start_time, 2), "sec.")

+-------------+----+-----+-----+-----+------------------+----+
|           id|year|month|max_t|min_t|            t_diff|rank|
+-------------+----+-----+-----+-----+------------------+----+
|1571958030336|2016|   10| 10.8| -7.7|              18.5|   1|
|  77309411328|2016|   10| 19.2|  1.0|              18.2|   2|
| 695784701957|2016|   10| 15.2| -1.1|              16.3|   3|
| 369367187456|2016|   10| 21.1|  4.9|16.200000000000003|   4|
|  77309411335|2016|   10| 20.7|  5.8|14.899999999999999|   5|
|1520418422787|2016|   10| 20.7|  5.8|14.899999999999999|   6|
|1013612281860|2016|   10| 19.2|  4.5|              14.7|   7|
| 137438953473|2016|   10| 22.1|  7.7|14.400000000000002|   8|
|1108101562369|2016|   10| 16.3|  2.1|14.200000000000001|   9|
| 335007449088|2016|   10| 17.3|  3.8|              13.5|  10|
| 824633720837|2017|    8| 21.8| 13.3|               8.5|   1|
| 996432412678|2017|    8| 26.0| 17.5|               8.5|   2|
| 730144440321|2017|    8| 26.0| 17.5|               8.

In [27]:
result.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Filter (rank#4755 < 11)
   +- Window [row_number() windowspecdefinition(year#11, month#12, t_diff#3947 DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rank#4755], [year#11, month#12], [t_diff#3947 DESC NULLS LAST]
      +- WindowGroupLimit [year#11, month#12], [t_diff#3947 DESC NULLS LAST], row_number(), 10, Final
         +- Sort [year#11 ASC NULLS FIRST, month#12 ASC NULLS FIRST, t_diff#3947 DESC NULLS LAST], false, 0
            +- Exchange hashpartitioning(year#11, month#12, 200), ENSURE_REQUIREMENTS, [plan_id=1798]
               +- WindowGroupLimit [year#11, month#12], [t_diff#3947 DESC NULLS LAST], row_number(), 10, Partial
                  +- Sort [year#11 ASC NULLS FIRST, month#12 ASC NULLS FIRST, t_diff#3947 DESC NULLS LAST], false, 0
                     +- Filter (isnotnull(t_diff#3947) AND (t_diff#3947 > 0.0))
                        +- HashAggregate(keys=[id#6, year#11, m